In [2]:
# import os
# from PIL import Image
# from pillow_heif import register_heif_opener
# from PIL import ImageFile

# # Cho phép xử lý tệp bị lỗi hoặc không đầy đủ
# ImageFile.LOAD_TRUNCATED_IMAGES = True

# # Đăng ký HEIF/HEIC opener với Pillow
# register_heif_opener()

# def convert_heic_to_png(heic_file, png_file):
#     """Chuyển đổi tệp HEIC sang PNG"""
#     try:
#         image = Image.open(heic_file)  # Mở tệp HEIC
#         image.save(png_file, "PNG")   # Lưu dưới dạng PNG
#         print(f"Converted {heic_file} to {png_file}")
#     except Exception as e:
#         print(f"Failed to convert {heic_file}: {e}")

# # Thư mục chứa tệp HEIC
# file_path = "./DATA"

# # Duyệt qua tất cả các tệp trong thư mục
# for file in os.listdir(file_path):
#     if file.lower().endswith(".heic"):  # Kiểm tra đuôi tệp
#         heic_file = os.path.join(file_path, file)
#         png_file = os.path.splitext(heic_file)[0] + ".png"

#         # Kiểm tra nếu file PNG đã tồn tại
#         if os.path.exists(png_file):
#             print(f"Skipping {file}, {png_file} already exists.")
#             continue

#         # Chuyển đổi nếu file PNG chưa tồn tại
#         convert_heic_to_png(heic_file, png_file)
#     else:
#         print(f"Skipping {file}, not a HEIC file.")

# print("Done")


In [2]:
import os
import cv2
import numpy as np
import torch
import shutil
from mmocr.apis.inferencers import MMOCRInferencer
from mmocr.utils import poly2bbox

import warnings
warnings.filterwarnings("ignore")


In [3]:
# Cấu hình
input_dir = "./DATA"  # Thư mục chứa ảnh sản phẩm
output_dir = "./output/"  # Thư mục lưu kết quả
os.makedirs(output_dir, exist_ok=True)

In [4]:
# Khởi tạo MMOCR Inferencer
mmocr_inferencer = MMOCRInferencer(
    det="mmocr_dev/configs/textdet/dbnetpp/dbnetpp_swinv2_base_w16_in21k.py",  # Mô hình phát hiện
    det_weights="checkpoints/mmocr/db_swin_mix_pretrain.pth",  # Checkpoint phát hiện
    rec="mmocr_dev/configs/textrecog/abinet/abinet_20e_st-an_mj.py",  # Mô hình nhận dạng
    rec_weights="checkpoints/mmocr/abinet_20e_st-an_mj_20221005_012617-ead8c139.pth",  # Checkpoint nhận dạng
    device="cpu",  # Hoặc "cuda" nếu bạn có GPU
)
print("MMOCR Inferencer initialized successfully.")


Loads checkpoint by local backend from path: checkpoints/mmocr/db_swin_mix_pretrain.pth
11/20 11:07:02 - mmengine - INFO - Delete `relative_position_index` and `relative_coords_table` since we always re-init these params according to the `window_size`, which might cause unwanted but unworried warnings when loading checkpoint.
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.stages.0.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.0.attn.w_msa.relative_position_index, backbone.stages.0.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.1.attn.w_msa.relative_position_index, backbone.stages.1.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.0.attn.w_msa.relative_position_index, backbone.stages.1.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.1.attn.w_msa.relative_position_index, backbone.stages.2.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.2.blocks.0

In [5]:
# Hàm trích xuất văn bản từ ảnh và lưu vào thư mục đầu ra
def extract_text(image_path, output_dir, mmocr_inferencer):
    # Chạy inference và lưu ảnh đã xử lý vào output_dir
    print(f"Processing: {image_path}")
    results = mmocr_inferencer(image_path, save_vis=True, out_dir=output_dir)

    # Lấy kết quả văn bản từ mô hình
    texts = []
    for prediction in results["predictions"]:
        for text in prediction["rec_texts"]:
            texts.append(text)

    # Tạo tên file .txt dựa trên tên ảnh gốc
    output_text_file = os.path.join(
        output_dir,
        os.path.basename(image_path).replace(".jpg", ".txt").replace(".png", ".txt"),
    )

    # Lưu file văn bản vào thư mục đầu ra
    with open(output_text_file, "w", encoding="utf-8") as f:
        f.write("\n".join(texts))

    print(f"Extracted texts saved to: {output_text_file}")
    return texts


# Chạy trên tất cả ảnh trong thư mục đầu vào và lưu vào thư mục đầu ra
image_files = [
    os.path.join(input_dir, f)
    for f in os.listdir(input_dir)
    if f.endswith((".jpg", ".png"))
]

# Xoá tất cả file trong thư mục đầu ra
for f in os.listdir(output_dir):
    file_path = os.path.join(output_dir, f)
    if os.path.isfile(file_path) or os.path.islink(file_path):
        os.remove(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)

for image_file in image_files:
    # Chỉ xử lý 5 ảnh đầu tiên
    if len(os.listdir(output_dir)) >= 5:
        break
    extract_text(image_file, output_dir, mmocr_inferencer)

Processing: ./DATA\IMG_3455.png


Extracted texts saved to: ./output/IMG_3455.txt
Processing: ./DATA\IMG_3456.png


Extracted texts saved to: ./output/IMG_3456.txt
Processing: ./DATA\IMG_3457.png


Extracted texts saved to: ./output/IMG_3457.txt
Processing: ./DATA\IMG_3458.png


Extracted texts saved to: ./output/IMG_3458.txt
